# Time Series Extractor, Slicer and Store Introduction

This notebook intends to show you different queries that can be done with the `TimeSeriesExtractor` class that `soam` provides, such as:
- Simple extract * query.
- Categorical data filters or conditions.
- Temporal data filters or conditions.
- Aggregated fields.

At the same time, we provide a brief introduction to the `Slicer` class that `soam` provides to generate slices of your `DataFrame`. Finally, we show how to persist the results by using the `Store` class from the `soam` workflow.

## Database used
We will be using a database that stores information on Apple's and Tesla's stocks over a period of time.

## Setup

The setup consists of:
- Import the needed dependencies.
- Establish the connection with our database.

In [1]:
from soam.workflow.time_series_extractor import TimeSeriesExtractor
from muttlib.dbconn import get_client

In [2]:
sqlite_cfg = {
    "db_type": "sqlite",
    "database": "soam_quickstart.db"
}

sqlite_client = get_client(sqlite_cfg)[1]

## Time Series Extractor

- Instantiate our extractor object.

> To build a query you should create a `dictionary` of `{str: obj}` that will be later used for the extraction by executing the `extractor.run` method.

In [3]:
extractor = TimeSeriesExtractor(db=sqlite_client, table_name='stock')

### Simple extract * query

Simple query, just retrieving all the data from the database.

In [10]:
query={
    'columns': '*'
}

print('Query generated: ', extractor.build_query(columns= '*')[0])

Query generated:  
          
          SELECT *
          FROM stock
          
          
          
          
          
        


In [11]:
df = extractor.run(build_query_kwargs = query)
df.head()

id        date symbol  avg_num_trades  avg_price
0   1  2021-03-01   AAPL         80000.0      125.0
1   2  2021-03-02   AAPL         70000.0      126.0
2   3  2021-03-03   AAPL         80000.0      123.0
3   4  2021-03-04   AAPL         70000.0      121.0
4   5  2021-03-05   AAPL         80000.0      119.0

### Categorical data filters or conditions
Adding some filters and conditionals:
- Querying only a subset of the columns.
- Renaming some columns with aliases.
- Filtering data by just retrieving Apple's stock valuations.

In [12]:
query={
    'columns': ['date', 'symbol', 'avg_price AS Valuation'],
    'extra_where_conditions': ["symbol = 'AAPL'"]
}

print('Query generated: ', extractor.build_query(columns = ['date', 'symbol', 'avg_price AS Valuation'],
                                                 extra_where_conditions = ["symbol = 'AAPL'"])[0])

Query generated:  
          
          SELECT date, symbol, avg_price AS Valuation
          FROM stock
          
          
          WHERE symbol = 'AAPL'
          
          
          
          
        


In [13]:
df = extractor.run(build_query_kwargs = query)
df.head()

date symbol  Valuation
0  2021-03-01   AAPL      125.0
1  2021-03-02   AAPL      126.0
2  2021-03-03   AAPL      123.0
3  2021-03-04   AAPL      121.0
4  2021-03-05   AAPL      119.0

### Temporal data filters or conditions
Adding some extra filters and sorting results:
- Filtering data by certain dates.
- Sorting results based on their dates in ascending order.

In [14]:
query={
    'columns': ['date', 'symbol', 'avg_price AS Valuation'],
    'timestamp_col': 'date',
    'start_date': "2021-03-01",
    'end_date': "2021-03-20",
    'extra_where_conditions': ["symbol = 'AAPL'"],
    'order_by': ["date ASC"]
}

print('Query generated: ', extractor.build_query(columns = ['date', 'symbol', 'avg_price AS Valuation'],
                                                 timestamp_col = 'date',
                                                 start_date = "2021-03-01",
                                                 end_date = "2021-03-20",
                                                 extra_where_conditions = ["symbol = 'AAPL'"],
                                                 order_by = ["date ASC"])[0])

Query generated:  
          
          SELECT date, symbol, avg_price AS Valuation
          FROM stock
          
          
          WHERE date >= '2021-03-01' AND date <= '2021-03-20' AND symbol = 'AAPL'
          
          
          
          
          ORDER BY date ASC
          
        


In [15]:
df = extractor.run(build_query_kwargs = query)
df.head()

date symbol  Valuation
0  2021-03-01   AAPL      125.0
1  2021-03-02   AAPL      126.0
2  2021-03-03   AAPL      123.0
3  2021-03-04   AAPL      121.0
4  2021-03-05   AAPL      119.0

### Aggregated Fields I

Adding some aggregated data.
- Multiply the average valuation with the amount of trades to obtain the transactional volume per day.
- Group by date and symbol by using the dimensions method.
- Filter by a certain level of volume by using the having method.
- Sort results by volume in descending order.

In [17]:
query={
    'columns': ['date', 'symbol', 'avg_num_trades * avg_price AS Volume'],
    'dimensions': ['date','symbol'],
    'timestamp_col': 'date',
    'start_date': "2021-03-01",
    'end_date': "2021-03-20",
    'order_by': ["Volume DESC"],
    'extra_having_conditions': ['Volume > 1000000']
}

print('Query generated: ', extractor.build_query(columns = ['date', 'symbol', 'avg_num_trades * avg_price AS Volume'],
                                                 dimensions = ['date','symbol'],
                                                 timestamp_col = 'date',
                                                 start_date = "2021-03-01",
                                                 end_date = "2021-03-20",
                                                 order_by = ["Volume DESC"],
                                                 extra_having_conditions = ['Volume > 1000000'])[0])

Query generated:  
          
          SELECT date, symbol, avg_num_trades * avg_price AS Volume
          FROM stock
          
          
          WHERE date >= '2021-03-01' AND date <= '2021-03-20'
          
          
          GROUP BY date, symbol
          
          
          HAVING Volume > 1000000
          
          
          ORDER BY Volume DESC
          
        


In [18]:
df = extractor.run(build_query_kwargs = query)
df.head()

date symbol      Volume
0  2021-03-19   AAPL  16320000.0
1  2021-03-20   AAPL  16320000.0
2  2021-03-17   AAPL  15180000.0
3  2021-03-16   AAPL  14280000.0
4  2021-03-18   AAPL  14190000.0

### Aggregated Fields II

Adding some aggregated data.
- Retrieve the day with the biggest transactional volume for each company.

In [19]:
query={
    'columns': ['date', 'symbol', 'max(avg_num_trades * avg_price) AS Max_Volume'],
    'dimensions': ['symbol'],
    'order_by': ["Max_Volume DESC"],
}

print('Query generated: ', extractor.build_query(columns = ['date', 'symbol', 'max(avg_num_trades * avg_price) AS Max_Volume'],
                                                 dimensions = ['symbol'],
                                                 order_by = ["Max_Volume DESC"]
                                                )[0])

Query generated:  
          
          SELECT date, symbol, max(avg_num_trades * avg_price) AS Max_Volume
          FROM stock
          
          
          
          GROUP BY symbol
          
          
          
          ORDER BY Max_Volume DESC
          
        


In [20]:
df = extractor.run(build_query_kwargs = query)
df.head()

date symbol  Max_Volume
0  2021-03-22   AAPL  21300000.0
1  2021-03-08   TSLA  10324000.0

## Slicer

Slice a dataframe upon given dimensions.

#### 1. Instantiate the class:

Slice the incoming data upon the given dimensions

        Parameters
        ----------
        dimensions:
            str or list of str labels of categorical columns to slices
        metrics:
            str or list of str labels of metrics columns to slices
        ds_col:
            str of datetime column
        keeps:
            str or list of str labels of columns to keep.
            
            
            
#### 2. Execute the .run method:

Slice the given dataframe with the dimensions setted.

        Parameters
        ----------
        raw_df
            A pandas DataFrame containing the raw data to slice

        Returns
        -------
        list[pd.DataFrame]
            DataFrame containing the sliced dataframes.

### Use case
Imagine we want to generate a unique `DataFrame` for each dimension value with each metric. 

In our case, this means 4 dataframes since we have `Apple` and `Tesla` from the `Symbol` dimension and two metrics: `avg_num_trades` and `avg_price`. This will result on the following combinations:
1. Apple's average amount of trades per day.
2. Apple's average price per day.
3. Tesla's average amount of trades per day.
4. Tesla's average price per day.

*We will be using the same database as before but retrieving all the data from the table to generate our `DataFrame`.*

In [21]:
from soam.workflow.slicer import Slicer

In [23]:
query={
    'columns': '*'
}

df = extractor.run(build_query_kwargs = query)

df.head()

id        date symbol  avg_num_trades  avg_price
0   1  2021-03-01   AAPL         80000.0      125.0
1   2  2021-03-02   AAPL         70000.0      126.0
2   3  2021-03-03   AAPL         80000.0      123.0
3   4  2021-03-04   AAPL         70000.0      121.0
4   5  2021-03-05   AAPL         80000.0      119.0

### 1. Instantiate the class:

> - dimensions: `symbol` 
> - metrics: `avg_num_trades` and `avg_price`
> - ds_col: `date`

In [24]:
slicer = Slicer(metrics=["avg_num_trades", "avg_price"], ds_col="date", dimensions=["symbol"])

### 2. Execute the .run method

In [25]:
apple_trades, apple_price, tesla_trades, tesla_price = slicer.run(df)

#### Apple's average amount of trades per day:

In [26]:
apple_trades.head()

date symbol  avg_num_trades
0  2021-03-01   AAPL         80000.0
1  2021-03-02   AAPL         70000.0
2  2021-03-03   AAPL         80000.0
3  2021-03-04   AAPL         70000.0
4  2021-03-05   AAPL         80000.0

#### Apple's average price per day:

In [27]:
apple_price.head()

date symbol  avg_price
0  2021-03-01   AAPL      125.0
1  2021-03-02   AAPL      126.0
2  2021-03-03   AAPL      123.0
3  2021-03-04   AAPL      121.0
4  2021-03-05   AAPL      119.0

#### Tesla's average amount of trades per day:

In [28]:
tesla_trades.head()

date symbol  avg_num_trades
22  2021-03-01   TSLA         60000.0
23  2021-03-02   TSLA         62000.0
24  2021-03-03   TSLA         64000.0
25  2021-03-04   TSLA         69000.0
26  2021-03-05   TSLA         80000.0

#### Tesla's average price per day

In [29]:
tesla_price.head()

date symbol  avg_price
22  2021-03-01   TSLA      105.0
23  2021-03-02   TSLA      104.0
24  2021-03-03   TSLA      101.0
25  2021-03-04   TSLA      108.0
26  2021-03-05   TSLA      115.0

## Store
A class to store results in a table.

#### 1. Instantiate the class:

Store given data into a DataBase

        Parameters
        ----------
        db_cli:
            BaseClient client.
        table:
            str of table to store in.
        extra_insert_args:
            dict extra arguments to insert data.
                   
            
#### 2. Execute the .run method:

Store given DataFrame.

        Parameters
        ----------
        df
            A pandas DataFrame to store.


### Use case
Imagine we want to persist the `DataFrame` created of Apple's average price per day on a specific table named `apple_stock_price` in the same `DataBase` we are working on.

In [30]:
from soam.workflow.store import Store

### 1. Instantiate the class:

> - db_cli: `sqlite_client` 
> - table: `apple_stock_price`

In [32]:
saver = Store(db_cli=sqlite_client, table='apple_stock_price')

### 2. Execute the .run method

In [33]:
saver.run(apple_price)

We can extract it with the `TimeSeriesExtractor` object to verify it was persisted successfully.

In [34]:
extractor = TimeSeriesExtractor(db=sqlite_client, table_name='apple_stock_price')
query={
    'columns': '*'
}

persisted_df = extractor.run(build_query_kwargs = query)

persisted_df.head()

date symbol  avg_price
0  2021-03-01   AAPL      125.0
1  2021-03-02   AAPL      126.0
2  2021-03-03   AAPL      123.0
3  2021-03-04   AAPL      121.0
4  2021-03-05   AAPL      119.0

Voila!